In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

resultDatasetPath = "../dataset/users_clean_with_indicators_with_outliers.h5"

In [2]:
%%time
dataseUsersCleanedPath = "../dataset/users_clean_with_outliers.h5"
df_users = pd.read_hdf(dataseUsersCleanedPath)

CPU times: user 73.8 ms, sys: 12.3 ms, total: 86.2 ms
Wall time: 103 ms


In [3]:
%%time
datasetTweetsCleanedPath = "../dataset/tweets_clean_with_outliers.h5"
df_tweets = pd.read_hdf(datasetTweetsCleanedPath)

CPU times: user 5.79 s, sys: 6.82 s, total: 12.6 s
Wall time: 13.4 s


In [25]:
df_users.head(100)

,name,lang,bot,created_at,statuses_count
id,,,,,
2353593986,lamonica raborn,en,1,2019-02-22 18:00:42,76.0
2358850842,lourie botton,en,0,2019-02-26 03:02:32,54.0
137959629,dadan syarifudin,en,1,2015-04-30 07:09:56,3.0
466124818,carletto focia,it,1,2017-01-18 02:49:18,50.0
2571493866,mbk ebook,en,0,2019-06-18 19:30:21,7085.0
...,...,...,...,...,...
2738679910,tewtruenixひfⅎ,en,0,2019-08-12 02:40:12,15101.0
467116841,gabriele danno,it,1,2017-01-19 04:08:16,61.0
51381838,ik,en,0,2014-06-29 07:57:03,231.0


In [30]:
df_users.index.unique().min()

678033

#### tweets_count: how many tweets were published by the user

In [22]:
df_tweets.user_id.value_counts()

-1.000000e+00    213715
 4.916306e+08      3926
 1.693275e+09      3906
 1.570298e+08      3901
 7.004532e+06      3897
                  ...  
 1.373570e+09         1
 6.000000e+01         1
 1.726000e+03         1
 8.490000e+02         1
 2.610000e+02         1
Name: user_id, Length: 12142, dtype: int64

In [34]:
counts

-1.000000e+00    213715
 4.916306e+08      3926
 1.693275e+09      3906
 1.570298e+08      3901
 7.004532e+06      3897
                  ...  
 1.373570e+09         1
 6.000000e+01         1
 1.726000e+03         1
 8.490000e+02         1
 2.610000e+02         1
Name: user_id, Length: 12142, dtype: int64

In [49]:
df_users_ind = df_users.copy()

In [50]:
df_counts = df_tweets.user_id.value_counts().to_frame("tweet_count")
df_counts

,tweet_count
-1.000000e+00,213715
4.916306e+08,3926
1.693275e+09,3906
1.570298e+08,3901
7.004532e+06,3897
...,...
1.373570e+09,1
6.000000e+01,1
1.726000e+03,1
8.490000e+02,1


In [54]:
df_users_ind = df_users_ind.merge(df_counts, how="inner", left_index=True, right_index=True)
df_users_ind

,name,lang,bot,created_at,statuses_count,tweet_count
2.353594e+09,lamonica raborn,en,1,2019-02-22 18:00:42,76.0,127
2.358851e+09,lourie botton,en,0,2019-02-26 03:02:32,54.0,116
1.379596e+08,dadan syarifudin,en,1,2015-04-30 07:09:56,3.0,3
4.661248e+08,carletto focia,it,1,2017-01-18 02:49:18,50.0,1378
2.571494e+09,mbk ebook,en,0,2019-06-18 19:30:21,7085.0,3506
...,...,...,...,...,...,...
2.911862e+09,madrid lae maika .,en,0,2019-11-29 13:16:02,1126.0,1066
1.378533e+09,clau sato,en,0,2018-04-27 03:01:58,3024.0,2003
1.269841e+08,alma leticia nuño,es,0,2015-03-29 17:01:24,6.0,6
2.383026e+09,minnie guadagno,en,1,2019-03-13 02:44:13,42.0,101


In [69]:
count_peruser = df_tweets.groupby(["user_id"]).sum()
count_peruser

/tmp/ipykernel_17951/3974280124.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  count_peruser = df_tweets.groupby(["user_id"]).sum()


,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions
user_id,,,,,,
-1.000000e+00,96712376,200931,1188367,31965,30259,93679
0.000000e+00,93,0,1,0,0,1
2.000000e+00,0,0,0,0,0,0
3.000000e+00,0,0,0,0,0,3
5.000000e+00,0,0,0,0,0,0
...,...,...,...,...,...,...
4.800000e+64,0,0,0,0,0,0
3.300000e+66,0,0,0,0,0,0
4.000000e+95,1,0,0,0,0,0


In [70]:
df_users_ind = df_users_ind.merge(count_peruser, how="inner", left_index=True, right_index=True)
df_users_ind

,name,lang,bot,created_at,statuses_count,tweet_count,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions
2.353594e+09,lamonica raborn,en,1,2019-02-22 18:00:42,76.0,127,5,0,5,13,0,35
2.358851e+09,lourie botton,en,0,2019-02-26 03:02:32,54.0,116,3,0,6,3,0,41
1.379596e+08,dadan syarifudin,en,1,2015-04-30 07:09:56,3.0,3,0,0,0,0,0,0
4.661248e+08,carletto focia,it,1,2017-01-18 02:49:18,50.0,1378,1187,0,255,111,35,9
2.571494e+09,mbk ebook,en,0,2019-06-18 19:30:21,7085.0,3506,61,0,223,454,3,1832
...,...,...,...,...,...,...,...,...,...,...,...,...
2.911862e+09,madrid lae maika .,en,0,2019-11-29 13:16:02,1126.0,1066,6558313,0,63,227,31,829
1.378533e+09,clau sato,en,0,2018-04-27 03:01:58,3024.0,2003,792490,0,443,252,201,1087
1.269841e+08,alma leticia nuño,es,0,2015-03-29 17:01:24,6.0,6,0,0,0,2,4,2
2.383026e+09,minnie guadagno,en,1,2019-03-13 02:44:13,42.0,101,7,0,9,1,0,38
